Os treinamentos das redes MLP e Resnet foram feitos com um batch de 64 imagens por epoca e 10 epocas de treinamento com o dataset Cifar10

#CNN

In [1]:
'''For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        #ultima camada de features
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        #inserir uma nova camada aqui
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
#from loader import MultiFolderLoader
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
#from torch.utils.tensorboard import SummaryWriter

#from auto_augment import AutoAugment
#from auto_augment import Cutout

import os
import argparse

import numpy as np
from torch.autograd import Variable



#from PreResNet import * 

batch_size = 64
lr = 0.02


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),        
#    AutoAugment(),
#    Cutout(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  #,
])

trainset = datasets.CIFAR10(root='~/data', train=True, download=True, transform=transform_train) #'./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root='~/data', train=False, download=True, transform=transform_test)

trainloader  = DataLoader(trainset,  batch_size=batch_size, shuffle=True,  num_workers=3)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


# Model
print('==> Building model..')

net = ResNet18() 


net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True


Softmin = nn.Softmin()
Softmax = nn.Softmax()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9, weight_decay=5e-4)



# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    listSum = 0
  

    lr=0.02 
    if epoch >= 150:
        lr /= 10      
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr       

    #print("antes")
    for batch_idx, data in enumerate(trainloader, 0):
#    for batch_idx, (inputs, targets) in enumerate(trainloader):
#        print(batch_idx)
        inputs, targets = data 
        inputs, targets = inputs.to(device), targets.to(device).view(-1)
        optimizer.zero_grad()
        outputs = net(inputs)
        #print(outputs.size)
        batch_size = outputs.size()[0]
        lam = 2
        loss = criterion(outputs,targets)
        #print(loss)
 
        loss.backward()
        optimizer.step()   
        #train_loss += loss.data[0]  
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().float()# predicted.eq(targets).sum().item()
        acc = 100.*correct/total
    print('loss train')  
    print((train_loss/64))
    print('')
    print('---------------------------------------------------------------')
    print('Acc train')
    print(acc)
    print('')    
    print('---------------------------------------------------------------')

   

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader,0):
            inputs, targets = inputs.to(device),  targets.to(device).view(-1)
            outputs = net(inputs)
            batch_size = outputs.size()[0]

            loss = criterion(outputs,targets)
            
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).cpu().sum().float()
            acc = 100.*correct/total
        #    progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #                 % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    print('Loss test')  
    print(test_loss/64)
    print('')
    print('---------------------------------------------------------------')
    print('Acc test')
    print(acc)
    print('')    
    print('---------------------------------------------------------------')

    # Save checkpoint.
    acc = 100.*correct/total

    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

    
    print("best_acc: %f"%best_acc)
    print('---------------------------------------------------------------')

    

for epoch in range(start_epoch, start_epoch+10):
    train(epoch)
    test(epoch)
  

==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /root/data/cifar-10-python.tar.gz to /root/data
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


==> Building model..

Epoch: 0
loss train
18.872665376402438

---------------------------------------------------------------
Acc train
tensor(43.5720, device='cuda:0')

---------------------------------------------------------------
Loss test
2.837715294212103

---------------------------------------------------------------
Acc test
tensor(58.7000)

---------------------------------------------------------------
Saving..
best_acc: 58.700001
---------------------------------------------------------------

Epoch: 1
loss train
12.672741437330842

---------------------------------------------------------------
Acc train
tensor(63.1560, device='cuda:0')

---------------------------------------------------------------
Loss test
2.1522699454799294

---------------------------------------------------------------
Acc test
tensor(68.6600)

---------------------------------------------------------------
Saving..
best_acc: 68.660004
---------------------------------------------------------------


#MLP

In [12]:
import time
import numpy as np
import torch
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Constantes
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
COLOR_CHANNELS = 3
EPOCHS = 10
LEARNING_RATES = [.00001, 0.0001, 0.001, 0.01, 0.1]
KEEP_RATES = [.5, .65, .8]
MOMENTUM_RATES = [.25, .5, .75]
WEIGHT_DECAY_RATES = [.0005, .005, .05]
BATCH_SIZE = 64
BATCH_IMAGE_COUNT = 10000
TRAIN_BATCHS = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4"]
TEST_BATCHES = ["data_batch_5"]
CLASSES = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(CLASSES)
PLOT = False



class Net(torch.nn.Module):
    def __init__(self, n_hidden_nodes, n_hidden_layers, activation, keep_rate=0):
        super(Net, self).__init__()
        self.n_hidden_nodes = n_hidden_nodes
        self.n_hidden_layers = n_hidden_layers
        self.activation = activation
        if not keep_rate:
            keep_rate = 0.5
        self.keep_rate = keep_rate
        # Configure camadas da rede e adicione dropout
        self.fc1 = torch.nn.Linear(IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS,
                                   n_hidden_nodes)
        self.fc1_drop = torch.nn.Dropout(1 - keep_rate)
        if n_hidden_layers == 2:
            self.fc2 = torch.nn.Linear(n_hidden_nodes,
                                       n_hidden_nodes)
            self.fc2_drop = torch.nn.Dropout(1 - keep_rate)

        self.out = torch.nn.Linear(n_hidden_nodes, N_CLASSES)

    def forward(self, x):
        x = x.view(-1, IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS)
        if self.activation == "sigmoid":
            sigmoid = torch.nn.Sigmoid()
            x = sigmoid(self.fc1(x))
        elif self.activation == "relu":
            x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc1_drop(x)
        if self.n_hidden_layers == 2:
            if self.activation == "sigmoid":
                x = sigmoid(self.fc2(x))
            elif self.activation == "relu":
                x = torch.nn.functional.relu(self.fc2(x))
            x = self.fc2_drop(x)
        return torch.nn.functional.log_softmax(self.out(x))


def train(epoch, model, train_loader, optimizer, log_interval=100, cuda=None):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)

        pred = output.data.max(1)[1] # obter o índice da probabilidade de log máxima
        correct += pred.eq(target.data).cpu().sum()
        accuracy = 100. * correct / len(train_loader.dataset)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #if batch_idx % log_interval == 0:
            #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Accuracy: {}'.format(
                #epoch, batch_idx * len(data), len(train_loader.dataset),
                #100. * batch_idx / len(train_loader), loss.item(), accuracy))
    print('Epoch:')
    print(epoch)
    print('loss:')
    print(loss.item())
    print('acc:')
    print(accuracy)       

def validate(loss_vector, accuracy_vector, model, validation_loader, cuda=None):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        val_loss += torch.nn.functional.nll_loss(output, target).item()
        pred = output.data.max(1)[1] # obter o índice da probabilidade de log máxima
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)

    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

def main():
    cuda = torch.cuda.is_available()
    print('Using PyTorch version:', torch.__version__, 'CUDA:', cuda)

    transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                              shuffle=True, num_workers=0, pin_memory=False)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    validation_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                             shuffle=False, num_workers=0, pin_memory=False)

    hidden_nodes = 100
    layers = 1
    for i in range(1, len(LEARNING_RATES) + 1):
        model = Net(hidden_nodes, layers, "sigmoid")
        if cuda:
            model.cuda()
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATES[i-1])

        loss_vector = []
        acc_vector = []
        for epoch in range(1, EPOCHS + 1):
            train(epoch, model, train_loader, optimizer, cuda=cuda)
            validate(loss_vector, acc_vector, model, validation_loader, cuda=cuda)
            if epoch == 25:
                break

        # Plote perda de trem e precisão de validação versus épocas para cada taxa de aprendizado
        if PLOT:
            epochs = [i for i in range(1, 26)]
            plt.plot(epochs, acc_vector)

            plt.xlabel("Epochs")
            plt.ylabel("Accuracy with Sigmoid")
            plt.show()

            plt.plot(epochs, loss_vector)
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.show()

    # Repita usando RELU para ativação

    hidden_nodes = 100
    layers = 1
    start_time = time.time()
    for i in range(1, len(LEARNING_RATES)):
        model = Net(hidden_nodes, layers, "relu")
        if cuda:
            model.cuda()
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATES[i])

        loss_vector = []
        acc_vector = []
        for epoch in range(1, EPOCHS + 1):
            train(epoch, model, train_loader, optimizer, cuda=cuda)
            validate(loss_vector, acc_vector, model, validation_loader, cuda=cuda)
            if epoch == 25:
                break
        end_time = time.time() - start_time
        print("Total time", end_time)
        # Plote perda de trem e precisão de validação versus épocas para cada taxa de aprendizado
        if PLOT:
            epochs = [i for i in range(1, 26)]
            plt.plot(epochs, acc_vector)
            plt.xlabel("Epochs")
            plt.ylabel("Accuracy with RELU")
            plt.show()

            plt.plot(epochs, loss_vector)
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.show()

    # Experimentando com/diferentes parâmetros:

    hidden_nodes = 100
    layers = 1
    start_time = time.time()
    for i in range(1, len(KEEP_RATES) + 1):
        model = Net(hidden_nodes, layers, "relu", keep_rate=KEEP_RATES[i-1])
        if cuda:
            model.cuda()
        optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATES[2], momentum=MOMENTUM_RATES[1], weight_decay=WEIGHT_DECAY_RATES[0])

        loss_vector = []
        acc_vector = []
        for epoch in range(1, EPOCHS + 1):
            train(epoch, model, train_loader, optimizer, cuda=cuda)
            validate(loss_vector, acc_vector, model, validation_loader, cuda=cuda)
            if epoch == 20:
                break
        end_time = time.time() - start_time
        print("Total time", end_time)
        if PLOT:
            epochs = [i for i in range(1, 21)]
            plt.plot(epochs, acc_vector)
            plt.xlabel("Epochs")
            plt.ylabel("Accuracy with RELU")
            plt.show()

            plt.plot(epochs, loss_vector)
            plt.xlabel("Epochs")
            plt.ylabel("Loss")
            plt.show()



    # 2 camadas, 50 nós ocultos:
    hidden_nodes = 50
    layers = 2
    start_time = time.time()

    model = Net(hidden_nodes, layers, "relu", keep_rate=.8)
    if cuda:
        model.cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATES[2])

    loss_vector = []
    acc_vector = []
    for epoch in range(1, EPOCHS + 1):
        train(epoch, model, train_loader, optimizer, cuda=cuda)
        validate(loss_vector, acc_vector, model, validation_loader, cuda=cuda)
        if epoch == 30:
            break
    end_time = time.time() - start_time
    print("Total time", end_time)

    if PLOT:
        epochs = [i for i in range(1, 31)]
        plt.plot(epochs, acc_vector)

        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.show()

        plt.plot(epochs, loss_vector)
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.show()

if __name__ == '__main__':
    main()


Using PyTorch version: 1.10.0+cu111 CUDA: True
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:
1
loss:
2.263942241668701
acc:
tensor(9.9540)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:98: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Validation set: Average loss: 2.3261, Accuracy: 998/10000 (10%)

Epoch:
2
loss:
2.363898515701294
acc:
tensor(10.7920)

Validation set: Average loss: 2.2978, Accuracy: 1010/10000 (10%)

Epoch:
3
loss:
2.4997098445892334
acc:
tensor(11.7580)

Validation set: Average loss: 2.2777, Accuracy: 1416/10000 (14%)

Epoch:
4
loss:
2.2511863708496094
acc:
tensor(12.4080)

Validation set: Average loss: 2.2625, Accuracy: 1803/10000 (18%)

Epoch:
5
loss:
2.317038059234619
acc:
tensor(13.0360)

Validation set: Average loss: 2.2503, Accuracy: 2027/10000 (20%)

Epoch:
6
loss:
2.5666637420654297
acc:
tensor(13.8240)

Validation set: Average loss: 2.2400, Accuracy: 2153/10000 (22%)

Epoch:
7
loss:
2.3886101245880127
acc:
tensor(14.3720)

Validation set: Average loss: 2.2309, Accuracy: 2227/10000 (22%)

Epoch:
8
loss:
2.131693124771118
acc:
tensor(15.2180)

Validation set: Average loss: 2.2227, Accuracy: 2332/10000 (23%)

Epoch:
9
loss:
2.3924851417541504
acc:
tensor(15.5480)

Validation set: Average los

IndexError: ignored

#Resultado

Na nossa resnet obtivemos melhores resultados nas mesmas condicões comparados a nossa MLP, entretando na nossa cnn necessitamos de mais parametros para regular os pesos, mas de forma mais geral a nossa cnn possui melhores resultados
